In [1]:
%pip install -q scipy numpy quantiphy pandas control

Note: you may need to restart the kernel to use updated packages.


In [4]:
import scipy as sci
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import control
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
from quantiphy import Quantity
import cmath
from typing import Optional, List, Tuple

In [5]:
# Prints a given parameter with its unit and using multipliers
def print_param(name: str, value: float, unit: str = "") -> None:
	print(name + ": " + str(Quantity(value, unit)))


# Converts a log value to a linear value
def log_2_linear(gain: float) -> float:
	return 10.0 ** (gain / 20.0)


# Converts a linear value to a log value
def linear_2_log(gain: float) -> float:
	return 20.0 * np.log10(gain)


# Commercial EE values list
def get_EE_list(limit: int = 21, lower = 0, pack: Optional[ List[ float ] ] = None) -> List[ float ]:
	return [x * (10 ** i) for i in range(pack[1] if pack else lower, pack[0] if pack else limit) for x in [ 1.5e-12, 1.8e-12, 2.2e-12, 2.7e-12, 3.3e-12, 4.7e-12, 5.1e-12, 6.8e-12, 7.5e-12, 8.2e-12, 9.1e-12, 10e-12 ]]


def get_EE_limit_from_value(value: float) -> List[ int ]:
	order = np.floor(np.log10(value))
	return [ int(13 + order), int((13 + order) - 1) ]


# Find the best fit for the closest comercial value in the EE list
def round_to_commercial(value: float) -> float:
	EE: List[ float ] = get_EE_list()
	index: int = 0
	close: float = np.inf
	for i, val in enumerate(EE):
		diff = val - value
		if np.abs(diff) < close:
			close = np.abs(diff)
			index = i
	return EE[index]


In [13]:
# Oscilation frequency
fosc: float = 1400

# Angular velocity
wc: float = 2 * np.pi * fosc

# Searches for the best component fit
last_diff: int = 9999999
res: List[ float ] = []
for C in [ 2.5784e-9 ]:
	R = 1 / (2 * np.pi * np.sqrt(6) * C * fosc)
	RD = 2 * R
	RH = R / 2
	RF = 29 * RD
	R_commercial = round_to_commercial(R)
	RH_commercial = round_to_commercial(RH)
	RD_commercial = round_to_commercial(RD)
	RF_commercial = round_to_commercial(RF)
	total_diff = np.abs(R_commercial - R) + np.abs(RF_commercial - RF) + np.abs(RH_commercial - RH)
	if (total_diff < last_diff):
		last_diff = total_diff
		res = [ C, R, RD, RF, R_commercial, RD_commercial, RF_commercial ]

# Fetches saved value
[ C, R, RD, RF, R_commercial, RD_commercial, RF_commercial ] = res

print("------------------------------------------------------------------")
print("Parâmetros do circuito: ")
print_param("    - Ideal C", C, "F")
print_param("    - Ideal R", R, "Ω")
print_param("    - Ideal RD", RD, "Ω")
print_param("    - Ideal RF", RF, "Ω")
print()
print_param("    - Comercial C", C, "F")
print_param("    - Comercial R", R_commercial, "Ω")
print_param("    - Comercial RD", RD_commercial, "Ω")
print_param("    - Comercial RF", RF_commercial, "Ω")
print("------------------------------------------------------------------")
print("Parâmetros para o LTSpice: ")
print("    - Ideal:")
print(f"\t.param C={Quantity(C)} R={Quantity(R)} RD={Quantity(RD)} RF={Quantity(RF)}")
print("    - Comercial:")
print(f"\t.param C={Quantity(C)} R={Quantity(R_commercial)} RD={Quantity(RD_commercial)} RF={Quantity(RF_commercial)}")

FC_commercial = 1 / (np.pi * 2 * C * R_commercial * np.sqrt(6))
FC_commercial_min = 1 / (np.pi * 2 * (C * 1.1) * (R_commercial * 1.05) * np.sqrt(6))
FC_commercial_max = 1 / (np.pi * 2 * (C * 0.9) * (R_commercial * 0.95) * np.sqrt(6))
print("------------------------------------------------------------------")
print_param("Valor de frequência comercial      ", FC_commercial, "Hz")
print_param("Valor de frequência comercial (MIN)", FC_commercial_min, "Hz")
print_param("Valor de frequência comercial (MAX)", FC_commercial_max, "Hz")
print("------------------------------------------------------------------")

------------------------------------------------------------------
Parâmetros do circuito: 
    - Ideal C: 2.5784 nF
    - Ideal R: 18 kΩ
    - Ideal RD: 35.999 kΩ
    - Ideal RF: 1.044 MΩ

    - Comercial C: 2.5784 nF
    - Comercial R: 18 kΩ
    - Comercial RD: 33 kΩ
    - Comercial RF: 1 MΩ
------------------------------------------------------------------
Parâmetros para o LTSpice: 
    - Ideal:
	.param C=2.5784n R=18k RD=35.999k RF=1.044M
    - Comercial:
	.param C=2.5784n R=18k RD=33k RF=1M
------------------------------------------------------------------
Valor de frequência comercial      : 1.4 kHz
Valor de frequência comercial (MIN): 1.2121 kHz
Valor de frequência comercial (MAX): 1.6374 kHz
------------------------------------------------------------------
